# پیاده سازی

In [ ]:
"""
یک ماژول برای پیاده‌سازی الگوریتم یادگیری گرادیان کاهش تصادفی برای یک شبکه عصبی پیش‌خور.
گرادیان‌ها با استفاده از روش انتشار معکوس محاسبه می‌شوند.
"""

# کتابخانه‌های استاندارد
import random

# کتابخانه‌های شخص ثالث
import numpy as np

class Network(object):
    """کلاس شبکه عصبی"""

    def __init__(self, sizes):
        """مقداردهی اولیه شبکه عصبی
        
        پارامترها:
        sizes -- لیستی که تعداد نورون‌های هر لایه را مشخص می‌کند.
                مثال: [2, 3, 1] برای شبکه‌ای با 2 نورون ورودی،
                3 نورون در لایه پنهان و 1 نورون در لایه خروجی
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        # مقداردهی تصادفی بایاس‌ها با توزیع نرمال
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # مقداردهی تصادفی وزن‌ها با توزیع نرمال
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """محاسبه خروجی شبکه برای ورودی داده شده
        
        پارامترها:
        a -- ورودی شبکه
        
        برگشت:
        خروجی شبکه
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
            
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """آموزش شبکه با گرادیان کاهش تصادفی روی دسته‌های کوچک
        
        پارامترها:
        training_data -- لیست داده‌های آموزشی به صورت (ورودی, خروجی مطلوب)
        epochs -- تعداد دوره‌های آموزشی
        mini_batch_size -- اندازه هر دسته کوچک
        eta -- نرخ یادگیری
        test_data -- داده‌های آزمون (اختیاری)
        """
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            # ایجاد دسته‌های کوچک
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            # آموزش روی هر دسته کوچک
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            # نمایش پیشرفت آموزش
            if test_data:
                print("دوره {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("دوره {0} تکمیل شد".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """به‌روزرسانی وزن‌ها و بایاس‌ها برای یک دسته کوچک
        
        پارامترها:
        mini_batch -- یک دسته کوچک از داده‌های آموزشی
        eta -- نرخ یادگیری
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            # محاسبه گرادیان‌ها با انتشار معکوس
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        # به‌روزرسانی وزن‌ها و بایاس‌ها
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """الگوریتم انتشار معکوس برای محاسبه گرادیان‌ها
        
        پارامترها:
        x -- ورودی آموزشی
        y -- خروجی مطلوب
        
        برگشت:
        یک تاپل (nabla_b, nabla_w) که گرادیان‌های تابع هزینه را نشان می‌دهد
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # مرحله پیش‌خور
        activation = x
        activations = [x]  # ذخیره فعال‌سازی‌های هر لایه
        zs = []  # ذخیره ورودی‌های هر لایه قبل از اعمال تابع فعال‌سازی
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # مرحله پس‌خور
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # محاسبه گرادیان‌ها برای لایه‌های قبلی
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """ارزیابی عملکرد شبکه روی داده آزمون
        
        پارامترها:
        test_data -- داده‌های آزمون
        
        برگشت:
        تعداد پیش‌بینی‌های صحیح
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """محاسبه مشتق تابع هزینه
        
        پارامترها:
        output_activations -- فعال‌سازی‌های لایه خروجی
        y -- خروجی مطلوب
        
        برگشت:
        مشتق تابع هزینه نسبت به فعال‌سازی‌های خروجی
        """
        return (output_activations-y)

# توابع کمکی
def sigmoid(z):
    """تابع فعال‌سازی سیگموئید"""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """مشتق تابع سیگموئید"""
    return sigmoid(z)*(1-sigmoid(z))

In [15]:
a = Network([10,5,1])
input = np.array( [1,1,1,1,1,2,2,2,2,2] , ndmin=2 ).T
print(input)
a.feedforward(input)

[[1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]]
a.shape: (10, 1)
w.shape: (5, 10)
output.shape: (5, 1)
a.shape: (5, 1)
w.shape: (1, 5)
output.shape: (1, 1)
a.shape: (1, 1)


array([[0.87953963]])